## Загрузка и подготовка данных

In [73]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split 

In [74]:
from numpy.random import RandomState
random = 12345
state = RandomState(random) 

In [75]:
# сделаем так, чтобы jupyter не выдавал ошибки
import warnings
warnings.filterwarnings("ignore")

In [76]:
data_0 = pd.read_csv('/datasets/geo_data_0.csv')
data_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [77]:
data_1 = pd.read_csv('/datasets/geo_data_1.csv')
data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [78]:
data_2 = pd.read_csv('/datasets/geo_data_2.csv')
data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


В данных содержится информация о геологоразведке в трёх регионах. Согласно документации:
* id — уникальный идентификатор скважины;
* f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
* product — объём запасов в скважине (тыс. баррелей).

Пропусков в данных не обнаружено. 

Посмотрим на сами таблицы:

In [79]:
display(data_0.head(), data_1.head(), data_2.head())

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


Столбец с идентификатором скважины нам не пригодится, поэтому его удалим. 

In [80]:
def id_drop(data):
    data.drop(['id'], axis=1, inplace=True)
    return data

data_list = [data_0, data_1, data_2]

for i in data_list:
    id_drop(i)

In [81]:
display(data_0.head(), data_1.head(), data_2.head())

,f0,f1,f2,product
0,0.705745,-0.497823,1.221170,105.280062
1,1.334711,-0.340164,4.365080,73.037750
2,1.022732,0.151990,1.419926,85.265647
3,-0.032172,0.139033,2.978566,168.620776
4,1.988431,0.155413,4.751769,154.036647


,f0,f1,f2,product
0,-15.001348,-8.276000,-0.005876,3.179103
1,14.272088,-3.475083,0.999183,26.953261
2,6.263187,-5.948386,5.001160,134.766305
3,-13.081196,-11.506057,4.999415,137.945408
4,12.702195,-8.147433,5.004363,134.766305


,f0,f1,f2,product
0,-1.146987,0.963328,-0.828965,27.758673
1,0.262778,0.269839,-2.530187,56.069697
2,0.194587,0.289035,-5.586433,62.871910
3,2.236060,-0.553760,0.930038,114.572842
4,-0.515993,1.716266,5.899011,149.600746


Математически опишем данные:

In [82]:
display(data_0.describe(), data_1.describe(), data_2.describe())

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


Можем заметить, что объёмы запасов во втором регионе сильно отличаются в распределении, медиане и среднем - их меньше. Первый и третий регион почти совпадают по цифрам. Стандартное отклонение во всех трех талицах почти одинаковое. Разброс данных небольшой, поэтому нам не нужно делать масштабирование. 

#### Определим признаки и целевой признак:

Признаки:
* f0, f1, f2

Целевой признак:
* product

Напишем функцию, чтобы делить данные на выборки и запишем их в переменные:

In [83]:
def split(data, target):
    features = data.drop([target], axis=1)
    target = data[target]
    
    features_train, features_valid, target_train, target_valid = train_test_split(features,
                                                    target,
                                                    test_size=0.25,
                                                    random_state=12345)
    return features_train, features_valid, target_train, target_valid

In [84]:
features_train_0, features_valid_0, target_train_0, target_valid_0 = split(data_0, 'product') 
    
features_train_1, features_valid_1, target_train_1, target_valid_1 = split(data_1, 'product') 
    
features_train_2, features_valid_2, target_train_2, target_valid_2 = split(data_2, 'product') 

##### Вывод

Данные подготовлены и изучены. Объединять таблицы не требуется, так как это усложнит работу. Больше всего запасов находится в третьем регионе - это можно понять по значениям всех четырех квартилей в сравнении с первым регионом, вторым по запасам нефти. Последним оказался второй регион, сильно отстающий от других. Возможно, в нем действительно не содержится больше запасов нефти, либо с данными что-то случилось. 

## Обучение и проверка модели

Для решения задачи будем использовать `линейную регрессию`, потому что она достаточно предсказуемая. Также из-за малого количества параметров она не склонна к переобучению. В качестве метри оценки модели будем использовать **RMSE**. 

In [85]:
def region_model(features_train, features_valid, target_train, target_valid, num):
    model = LinearRegression()
    model.fit(features_train, target_train)
    predicted = model.predict(features_valid)
    predicted_mean = predicted.mean()
    rmse = mean_squared_error(target_valid, predicted, squared=False)
    return f'RMSE модели региона {num} - {rmse}, Среднее предсказанное кол-во нефти {predicted_mean}', predicted

In [86]:
# запишем средние значения предсказаний в переменнные 
metrics_1, predicted_valid_0  = region_model(features_train_0, features_valid_0, target_train_0, target_valid_0, 1)
metrics_2, predicted_valid_1  = region_model(features_train_1, features_valid_1, target_train_1, target_valid_1, 2)
metrics_3, predicted_valid_2  = region_model(features_train_2, features_valid_2, target_train_2, target_valid_2, 3)

print(metrics_1)
print(metrics_2)
print(metrics_3)

RMSE модели региона 1 - 37.5794217150813, Среднее предсказанное кол-во нефти 92.59256778438035
RMSE модели региона 2 - 0.893099286775617, Среднее предсказанное кол-во нефти 68.728546895446
RMSE модели региона 3 - 40.02970873393434, Среднее предсказанное кол-во нефти 94.96504596800489


### Проверка на адекватность

In [87]:
def fact_checking(data, num, target):
    predictions = pd.Series(data[target].mean(), index=data[target].index)
    rmse = mean_squared_error(data[target], predictions)**0.5
    target_mean = data[target].mean()
    return f'Среднее реальное количество нефти в регионе {num} - {target_mean}, RMSE - {rmse}', target_mean

# записываем реальные значения средних показателей
valid_1, target_mean_1 = fact_checking(data_0, 1, 'product')
valid_2, target_mean_2 = fact_checking(data_1, 2, 'product')
valid_3, target_mean_3 = fact_checking(data_2, 3, 'product')

print(valid_1)
print(valid_2)
print(valid_3)

Среднее реальное количество нефти в регионе 1 - 92.50000000000001, RMSE - 44.2884696928441
Среднее реальное количество нефти в регионе 2 - 68.82500000000002, RMSE - 45.94419317228633
Среднее реальное количество нефти в регионе 3 - 95.00000000000004, RMSE - 44.74969731878746


##### Вывод
Видим, что сильно выбивается модель, обученная на данных по второму региону. Можем радоваться и без задумки брать ее для исследования прибыли, ведь метрики показывают, что она почти безупречно справляется с задачей. Но не все так просто, метрики оценки за первый и третий регион почти одинаковые, что наталкивает на мысль о проблемах в данных второго региона. Тем не менее, среднее количество нефти все 3 модели предсказали очень точно. Также мы записали значения реальных срежних показателей и показатели предсказаний в переменные, что пригодится нам в будущем. 

## Подготовка к расчёту прибыли

Запишем переменные с нужными нам значениями для расчетов. 

In [88]:
BUDGET = 10000000000
TOTAL_SPOTS = 500
TOP_SPOTS = 200
REVENUE = 450000
FLOOR_THRESHOLD = 0.025

Посчитаем, сколько тысяч баррелей должно приходится на точку, регион, а также среднюю доходность с одной скважины, чтобы определить параметры безубыточности.

In [89]:
av_spot_barrels = round((BUDGET/REVENUE)/TOP_SPOTS)
print(av_spot_barrels)

av_200_bareels = round(BUDGET / REVENUE)
print(av_200_bareels)

revenue_per_spot = round(BUDGET / TOP_SPOTS)
print(revenue_per_spot)

111
22222
50000000


##### Вывод

Итак, чтобы реализовать бюджет и не уйти в убыток, требуется:
* более 111 тыс баррелей в среднем с каждой скважины
* более 22222 тыс баррелей с 200 точек
* 50 млн рублей в среднем с каждой скважины

Видим, что средний запас для разработке новой скважины превышает средний запас в скважине в каждом регионе. Оценка рисков произведена. 

## Расчёт прибыли и рисков 

Напишем функцию для расчета прибыли. 

In [100]:
def estimation(target, predictions, count):
    preds_sorted = predictions.sort_values(ascending=False)
    selected = target[preds_sorted[:count].index]
    return  (REVENUE * selected.sum()) - BUDGET

Создадим функцию для бутстрепа. 

In [91]:
def bootstrap(target, predictions):
    values = []
    count = 0
    for i in range(1000):
        target_subsample = target.sample(n=TOTAL_SPOTS, replace=True, random_state=state)
        preds_subsample = predictions[target_subsample.index]
        esteem = estimation(target_subsample, preds_subsample, TOP_SPOTS)
        values.append(esteem)
        if esteem < 0:
            count += 1

    values = pd.Series(values)
    upper = values.quantile(q=0.975)
    lower = values.quantile(q=FLOOR_THRESHOLD)
    risk = count / 1000

    mean = values.mean()
    print("Средняя выручка:", mean)
    print('95%-квантиль:', upper)
    print("2.5%-квантиль:", lower)
    print('Процент риска {:.2%}'.format(risk))

И наконец функция преобразования признаков и предсказаний в Series. 

In [92]:
def result(target, predictions):
    target = pd.Series(target).reset_index(drop=True)
    predictions = pd.Series(predictions).reset_index(drop=True)
    return bootstrap(target, predictions)

In [101]:
result(target_valid_0, predicted_valid_0)

Средняя выручка: 603451756.5531541
95%-квантиль: 1279640911.4851658
2.5%-квантиль: 33048150.750933696
Процент риска 2.00%


In [102]:
result(target_valid_1, predicted_valid_1)

Средняя выручка: 647979547.1248649
95%-квантиль: 1207688415.5126615
2.5%-квантиль: 148579201.56019312
Процент риска 0.10%


In [103]:
result(target_valid_2, predicted_valid_2)

Средняя выручка: 591441867.061721
95%-квантиль: 1240709177.1626146
2.5%-квантиль: -43775594.1392455
Процент риска 3.40%


##### Вывод

Наиболее вероятно прибыльной оказался второй регион - об этом говорят данные. Он же и самый надежный - проецнт риска составляет 0.1. Первый и третий находятся на примерно одном уровне, однако первый является менее убыточным по квантилям и процент риска равен 2. Третий кажется самым ненадежным, и машина присвоиола ему 3.4% риск.  

## Общий вывод

Обучив модели на разных наборах данных, мы посмотрели на ее соответсвущие метрики и выяснили, что второй регион кажется самым надежным. Также мы посчитали среднюю выручку и точки безубыточности для всех регионах, где второй снова оказался в лидирующей позиции - его средняя выручка составила более **500 млн** рублей, минимальная выруча более **100 млн**, а максимальная более **850 млн**. Процент риска сосатвил **0.1%**. Остальные регионы показали себя хуже, с риском уйти в убыток. 

По результатам исследования делаем вывод, что `второй` регион является наиболее привлекательным к освоению. 